# Yelp dataset

## Import

In [ ]:
import pandas as pd
import os.path
import numpy as np
from tqdm import tqdm
import swifter
from sklearn.preprocessing import MinMaxScaler

## Clean and merge
Load the dataset in chunk to avoid memory overflow, delete not used column, and merge it with info about business activities

In [ ]:
def build_yelp_dataset():
    if os.path.exists('yelp dataset/yelp_dataset_merged.csv'):
        return
    df = pd.read_json('yelp dataset/yelp_academic_dataset_business.json', lines=True)
    df = df[df.is_open == 1] # consider only opened business
    df = df.reset_index(drop=True)
    df = df[['business_id', 'latitude', 'longitude']] # keep only this columns

    chunk_list = []  # append each chunk df here 
    # open the big review dataset in little chunk
    for count,chunk in enumerate(pd.read_json('yelp dataset/yelp_academic_dataset_review.json', lines=True, chunksize=500000)):
        chunk = chunk[['user_id', 'business_id', 'stars', 'date']]
        chunk = pd.merge(chunk, df, on='business_id') # merge business dataset and review dataset chunk
        chunk_list.append(chunk)
        print(f'processed {count + 1} chunks')
    df_concat = pd.concat(chunk_list)
    df_concat.to_csv('yelp dataset/yelp_dataset_merged.csv', index = False) # save dataset to CSV file

## Dataset preprocessing
We use the following contextual features: 
- year
- month
- day of the week
- week number
- longitude
- latitude

We extracted further contextual features from the date and location features: 
- season
- isHoliday
- isWeekend.

In [ ]:
df = pd.read_csv('yelp dataset/yelp_dataset_merged.csv')
df.rename(columns={'user_id': 'user', 'business_id': 'item'}, inplace=True)

# convert user and item IDs to int64
df.user = pd.factorize(df.user)[0]
df.item = pd.factorize(df.item)[0]
df

In [ ]:
# return a season from a date in the format d/m/Y
def season_from_date(date):
    year = str(date.year)
    seasons = {1: pd.date_range(start='21/03/'+year, end='20/06/'+year), # spring
               2: pd.date_range(start='21/06/'+year, end='22/09/'+year), # summer
               3: pd.date_range(start='23/09/'+year, end='20/12/'+year)} # autumn
    if date in seasons[1]:
        return 'spring'
    elif date in seasons[2]:
        return 'summer'
    elif date in seasons[3]:
        return 'autumn'
    else:
        return 'winter'

# faster version that consider genuary, february and march as winter and so on... you don't consider the day
def season_from_date_fast(date):
    seasons = np.arange(12)
    seasons = seasons.reshape(4, 3) # reshape to a 2D matrix
    i, j = np.where(seasons == date.month - 1) # get row where month appears
    return i[0] + 1 # 1 = winter, 2 = spring, 3 = summer, 4 = autumn

In [ ]:
df['date'] = pd.to_datetime(df['date']) # convert back to date
df['season'] = df['date'].swifter.apply(season_from_date_fast)

In [ ]:
df['weekday'] = df['date'].dt.dayofweek # get day of the week from date
df['weekend'] = (df['weekday'] == 6) | (df['weekday'] == 5) # if is weekend from week day
df['weeknumber'] = df['date'].dt.isocalendar().week # get week number

In [ ]:
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month

In [ ]:
df = df[['user', 'item', 'stars', 'month', 'year', 'weekday', 'weekend', 'weeknumber', 'season', 'latitude', 'longitude']]

## Encoding

- Categorical features **weekday**, **weekend** and **year** are encoded with one hot encoding.

- Cyclical features **month**, **week number** and **season** are encoded into two dimensions using a sine and cosine transformation:
$$
x_{sin} = \sin(\frac{2 \cdot \pi \cdot x}{max(x)}) \\
x_{cos} = \cos(\frac{2 \cdot \pi \cdot x}{max(x)})
$$
- Numeric features **latitude** and **longitude** are normalized


In [ ]:
def sin_cos_encoding(df, col, max_val):
    df[col + '_sin'] = np.sin(2 * np.pi * df[col]/max_val)
    df[col + '_cos'] = np.cos(2 * np.pi * df[col]/max_val)
    return df

In [ ]:
df = sin_cos_encoding(df, 'month', df.month.max()) # sin cos encoding for months
df = sin_cos_encoding(df, 'weeknumber', df.weeknumber.max()) # sin cos encoding for week number
df = sin_cos_encoding(df, 'season', df.season.max()) # sin cos encoding for seasons
df = df.drop(columns=['month', 'weeknumber', 'season'])

In [ ]:
# convert categorical data to one-hot encoding
for col in ['weekday', 'year']:
    df = pd.get_dummies(df, columns=[col], prefix = [col])

df['weekend'] = df.weekend.astype('UInt8')

In [ ]:
# min max normalization for latitude and longitude
mms = MinMaxScaler()
df[['latitude','longitude']] = mms.fit_transform(df[['latitude','longitude']])
df

In [ ]:
df.info(memory_usage='deep')
ax = df[:100000].plot.scatter('latitude', 'longitude').set_aspect('equal')

In [ ]:
save_folder = 'final datasets'
if not os.path.exists(save_folder):
    os.makedirs(save_folder)
df.to_csv(save_folder + '/yelp_final.csv', index = False) # save dataset to CSV file